In [21]:
# Import packages
import pandas as pd
import numpy as np
import datetime as dt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import matplotlib.pyplot as plt
import networkx as nx

import plotly.express as px
import plotly.graph_objects as go

In [2]:
data = pd.read_csv('C:/Users/migue/Desktop/data_preprocess.csv')

In [3]:
data.head()

Unnamed: 0  ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  \
0           1                16                  11              306   
1           3                16                  11              306   
2           5                16                  11              306   
3           9                16                  11              306   
4          10                16                  11              306   

   ProductName_ID  ProductPackSKU_ID  Point-of-Sale_ID        Date   Value  \
0             649               1970                 1  2017-03-04  1540.0   
1             649               1970                 1  2016-05-02  3080.0   
2             649               1970                 1  2016-10-24  1540.0   
3             649               1970                 1  2017-10-13  1620.0   
4             649               1970                 1  2017-10-14  1620.0   

   Day of the Week  Quarter  Units  
0                5        1    2.0  
1                0        2    4.0  
2                0        4    2.0  
3                4        4    2.0  
4                5        4    2.0

In [4]:
data['TID']=data['ProductFamily_ID'].astype(str)+data['Point-of-Sale_ID'].astype(str)+data['Date'].astype(str)

In [5]:
data.head(3)

Unnamed: 0  ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  \
0           1                16                  11              306   
1           3                16                  11              306   
2           5                16                  11              306   

   ProductName_ID  ProductPackSKU_ID  Point-of-Sale_ID        Date   Value  \
0             649               1970                 1  2017-03-04  1540.0   
1             649               1970                 1  2016-05-02  3080.0   
2             649               1970                 1  2016-10-24  1540.0   

   Day of the Week  Quarter  Units            TID  
0                5        1    2.0  1612017-03-04  
1                0        2    4.0  1612016-05-02  
2                0        4    2.0  1612016-10-24

In [6]:
#df=data.copy()

In [87]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91171152 entries, 0 to 91171151
Data columns (total 14 columns):
 #   Column              Dtype         
---  ------              -----         
 0   Unnamed: 0          int64         
 1   ProductFamily_ID    int64         
 2   ProductCategory_ID  int64         
 3   ProductBrand_ID     int64         
 4   ProductName_ID      int64         
 5   ProductPackSKU_ID   int64         
 6   Point-of-Sale_ID    int64         
 7   Date                datetime64[ns]
 8   Value               float64       
 9   Day of the Week     int64         
 10  Quarter             int64         
 11  Units               float64       
 12  TID                 object        
 13  Year                int32         
dtypes: datetime64[ns](1), float64(2), int32(1), int64(9), object(1)
memory usage: 9.2+ GB


In [8]:
#data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')

In [12]:
data['Year'] = data['Date'].astype(str).str[:4]

In [ ]:
data['Year']=data['Year'].astype(int)

In [88]:
df = data[['ProductFamily_ID','Point-of-Sale_ID','Year','Quarter','TID']].copy()

In [11]:
#df = data.groupby(['ProductFamily_ID','Point-of-Sale_ID','Year','Quarter','TID'],as_index=False)['Units'].mean()

In [89]:
df.sort_values('TID').head()

ProductFamily_ID  Point-of-Sale_ID  Year  Quarter              TID
24598288                10               100  2016        1  101002016-01-01
24497047                10               100  2016        1  101002016-01-02
24598289                10               100  2016        1  101002016-01-04
24288893                10               100  2016        1  101002016-01-04
24522230                10               100  2016        1  101002016-01-04

In [70]:
#data['Year']=data['Year'].astype(int)

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91171152 entries, 0 to 91171151
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   ProductFamily_ID  int64 
 1   Point-of-Sale_ID  int64 
 2   Year              int32 
 3   Quarter           int64 
 4   TID               object
dtypes: int32(1), int64(3), object(1)
memory usage: 3.1+ GB


In [95]:
orders = df[(df['Year']==2016)&(df['Quarter']==3)]

In [96]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5675074 entries, 16 to 91171138
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   ProductFamily_ID  int64 
 1   Point-of-Sale_ID  int64 
 2   Year              int32 
 3   Quarter           int64 
 4   TID               object
dtypes: int32(1), int64(3), object(1)
memory usage: 238.1+ MB


In [97]:
pt = pd.pivot_table(orders[['TID','ProductFamily_ID']], index='TID', columns='ProductFamily_ID', 
                    aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)
pt.head()

ProductFamily_ID   1    2    3    4    5    6    7    8    9    10  ...   12  \
TID                                                                 ...        
101002016-07-01   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.0   
101002016-07-02   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.0   
101002016-07-04   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.0   
101002016-07-05   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.0   
101002016-07-06   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.0   

ProductFamily_ID   13   14   15   16   17   18   19   20   21  
TID                                                            
101002016-07-01   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
101002016-07-02   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
101002016-07-04   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
101002016-07-05   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
101002016-07-06   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 21 columns]

In [98]:
frequent_itemsets = apriori(pt, min_support=0.01, use_colnames=True)
frequent_itemsets

support itemsets
0   0.072201      (1)
1   0.070583      (2)
2   0.047015      (3)
3   0.016740      (4)
4   0.060120      (5)
5   0.057629      (6)
6   0.048445      (7)
7   0.061747      (8)
8   0.072017      (9)
9   0.060653     (10)
10  0.033130     (11)
11  0.072048     (12)
12  0.031922     (13)
13  0.069081     (14)
14  0.070858     (15)
15  0.053688     (16)
16  0.070058     (18)
17  0.012150     (20)
18  0.071540     (21)
19  0.017168  (1, 12)
20  0.016742  (2, 21)

In [99]:
# Generate the association rules - by lift
rulesLift = association_rules(frequent_itemsets, metric="lift", min_threshold=0)
rulesLift.sort_values(by='confidence', ascending=False, inplace=True)
rulesLift.head()

antecedents consequents  antecedent support  consequent support   support  \
1        (12)         (1)            0.072048            0.072201  0.017168   
0         (1)        (12)            0.072201            0.072048  0.017168   
2         (2)        (21)            0.070583            0.071540  0.016742   
3        (21)         (2)            0.071540            0.070583  0.016742   

   confidence      lift  leverage  conviction  
1    0.238283  3.300274  0.011966    1.218037  
0    0.237778  3.300274  0.011966    1.217431  
2    0.237200  3.315613  0.011693    1.217173  
3    0.234027  3.315613  0.011693    1.213380